In [40]:
import pandas as pd
import numpy as np
import csv
from geopy.geocoders import Nominatim
import requests
import math
from tqdm import tqdm

In [2]:
df=pd.read_csv('Data/eq2015.csv')

In [3]:
num_rows = df.shape[0]
print("Number of rows:", num_rows)

Number of rows: 762106


In [4]:
# Add columns for the change in number of floors and height
df['floors_change']=df['count_floors_pre_eq']-df['count_floors_post_eq']
df['height_change']=df['height_ft_pre_eq']-df['height_ft_post_eq']

In [5]:
# Check for missing values in damage_grade
msdf = df[df['damage_grade'].isna()]
msdf[['building_id','damage_grade','technical_solution_proposed']].head()

,building_id,damage_grade,technical_solution_proposed
83766,203202000521,NaN,NaN
131558,212402000211,NaN,NaN
131579,212402000221,NaN,NaN
131627,212402000071,NaN,NaN
131629,212402000091,NaN,NaN


In [6]:
# Check for strange values which age is 999
msdf=df[df['age_building']==999]
msdf[['building_id', 'age_building']].head()

,building_id,age_building
2004,120306000071,999
6113,120903000031,999
12068,121701000241,999
12090,121701000461,999
12124,121701000811,999


In [7]:
# Check for strange values which pre_height is 99
msdf=df[df['height_ft_pre_eq']==99]
msdf[['building_id', 'height_ft_pre_eq','height_ft_post_eq']].head()

,building_id,height_ft_pre_eq,height_ft_post_eq
244076,231407000981,99,0
273673,234007000771,99,0
288816,235501001041,99,0
288817,235501001061,99,0
288820,235501001161,99,0


In [8]:
# Check for strange values which post_height is 99
msdf=df[df['height_ft_post_eq']==99]
msdf[['building_id', 'height_ft_pre_eq','height_ft_post_eq']].head()

,building_id,height_ft_pre_eq,height_ft_post_eq
394259,246707000201,22,99
405536,248107000051,99,99
405635,248107000871,99,99
599767,310402005321,99,99


In [9]:
# Check for strange values which increase the number of floors
msdf=df[df['floors_change']<0]
msdf[['building_id', 'count_floors_pre_eq', 'count_floors_post_eq', 'floors_change','height_ft_pre_eq','height_ft_post_eq','height_change','technical_solution_proposed']].head()

,building_id,count_floors_pre_eq,count_floors_post_eq,floors_change,height_ft_pre_eq,height_ft_post_eq,height_change,technical_solution_proposed
1639,120302000661,1,2,-1,19,19,0,Reconstruction
3136,120408000171,2,3,-1,12,12,0,No need
3146,120408000261,1,3,-2,20,20,0,No need
4564,120606000301,2,5,-3,13,13,0,No need
4992,120702000742,1,2,-1,22,22,0,No need


In [10]:
# Check for strange values which increase the height
msdf=df[df['height_change']<0]
msdf[['building_id', 'count_floors_pre_eq', 'count_floors_post_eq', 'floors_change','height_ft_pre_eq','height_ft_post_eq','height_change','technical_solution_proposed']].head()

,building_id,count_floors_pre_eq,count_floors_post_eq,floors_change,height_ft_pre_eq,height_ft_post_eq,height_change,technical_solution_proposed
729,120201000042,2,2,0,14,17,-3,Major repair
732,120201000071,2,2,0,14,17,-3,Major repair
736,120201000101,2,2,0,14,17,-3,Major repair
764,120201000341,2,2,0,14,17,-3,Major repair
926,120203000291,2,2,0,14,17,-3,Reconstruction


In [11]:
# Drop NaN values
df=df.dropna(subset=['damage_grade'])

# Drop age_building is 999
df=df.drop(df[df['age_building']==999].index)

# Drop height_ft_pre_eq is 99
df=df.drop(df[df['height_ft_pre_eq']==99].index)

# Drop height_ft_pre_eq is 99
df=df.drop(df[df['height_ft_pre_eq']==99].index)

# Drop negative change in height and floor
df=df.drop(df[df['height_change']<0].index)
df=df.drop(df[df['floors_change']<0].index)

In [12]:
df_level_1 = df[(df['damage_grade'] == 'Grade 1') | (df['damage_grade'] == 'Grade 2')]
df_level_2 = df[(df['damage_grade'] == 'Grade 3') | ((df['damage_grade'] == 'Grade 4') & (df['technical_solution_proposed'] != 'Reconstruction'))]
df_level_3 = df[(df['damage_grade'] == 'Grade 5') | ((df['damage_grade'] == 'Grade 4') & (df['technical_solution_proposed'] == 'Reconstruction'))]

In [13]:
# Check the data count
df.shape[0]-(df_level_1.shape[0]+ df_level_2.shape[0]+ df_level_3.shape[0])

0

In [14]:
df_level_1['damage_level'] = 1
df_level_2['damage_level'] = 2
df_level_3['damage_level'] = 3

C:\Users\49228\AppData\Local\Temp\ipykernel_12928\511933157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_level_1['damage_level'] = 1
C:\Users\49228\AppData\Local\Temp\ipykernel_12928\511933157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_level_2['damage_level'] = 2
C:\Users\49228\AppData\Local\Temp\ipykernel_12928\511933157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [15]:
# Merge the dataframes
df=pd.concat([df_level_1, df_level_2, df_level_3])

In [16]:
df.drop(['damage_grade','technical_solution_proposed','condition_post_eq'], axis=1, inplace=True)

In [22]:
# Return position of address
def geocoding(address):
    geolocator = Nominatim(user_agent='nepal_locator', timeout=None)
    geo = geolocator.geocode(address)
    if geo:
        crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}
    else:
        crd = {"lat": None, "lng": None}
    return crd

In [25]:
# Apply haversine formula to calculate distance
origin_lat = 28.147
origin_lmg = 84.708
def haversine(lat1, lon1, lat2, lon2):
    # 지구 반지름 (킬로미터)
    R = 6371.0
    
    # 위도와 경도를 라디안으로 변환
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1) 
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # 차이 계산
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # 해버사인 공식
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # 거리 계산
    distance = R * c # 미터 단위 
    distance *= 3.28084 # fit 단위로 변환 
    return distance

In [30]:
# Check address names
district_id_list = df['district_id'].unique()

url_1 = "https://eq2015.npc.gov.np/api/distributions?location_code="
url_2 = "&themes=indv_migration&depth=2"


# Create an empty dictionary
district_vcdmn_dict = {}
for district_i in district_id_list:
    df_vcdmn = df[df['district_id'] == district_i]
    df_vcdmn=df_vcdmn['vdcmun_id'].unique()
    for vcdmn_i in df_vcdmn:
        district_vcdmn_dict[(district_i, vcdmn_i)] = 0
# Add more key-value pairs as needed

for district_i in tqdm(district_id_list):
    url = url_1 + str(district_i) + url_2
    response = requests.get(url, verify=False)
    region = response.json()
    for j in range(len(region["data"]['indv_migration'])) :
        district_id = int(region["data"]['indv_migration'][j]["district_id"])
        vdcmun_id = int(region["data"]['indv_migration'][j]["vdcmun_id"])
        if (district_id, vdcmun_id) in district_vcdmn_dict.keys():
            if district_vcdmn_dict[(district_id, vdcmun_id)] == 0:
                district_name = region["data"]['indv_migration'][j]["district_name"]
                vdcmun_name = region["data"]['indv_migration'][j]["vdcmun_name"]
                vdcmun_name = vdcmun_name.split(" ")[0]
                address = vdcmun_name + " " + district_name
                crd = geocoding(address)
                if crd["lat"] == None or crd["lng"] == None:
                    address = district_name
                    crd = geocoding(address)
                crd=haversine(origin_lat, origin_lmg, float(crd["lat"]), float(crd["lng"]))
                district_vcdmn_dict[(district_id, vdcmun_id)] = crd


c:\Users\49228\anaconda3\envs\multilayer\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eq2015.npc.gov.np'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\49228\anaconda3\envs\multilayer\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eq2015.npc.gov.np'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\49228\anaconda3\envs\multilayer\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eq2015.npc.gov.np'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\U

In [31]:
# Add length from origin
df['distance'] = 0
for district_i, vcdmn_i in district_vcdmn_dict.keys():
    df.loc[(df['district_id'] == district_i) & (df['vdcmun_id'] == vcdmn_i), 'distance'] = district_vcdmn_dict[(district_i, vcdmn_i)]

C:\Users\49228\AppData\Local\Temp\ipykernel_12928\3928463087.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '653.3069264732525' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[(df['district_id'] == district_i) & (df['vdcmun_id'] == vcdmn_i), 'distance'] = district_vcdmn_dict[(district_i, vcdmn_i)]


In [33]:
# Drop loacation information
df.drop(['district_id','vdcmun_id','ward_id'], axis=1, inplace=True)

In [34]:
df.head()

,building_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,superstructure,floors_change,height_change,damage_level,distance
2,120101000031,1,1,20,384,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,has_superstructure_mud_mortar_stone,0,0,1,653.306926
3,120101000041,1,1,20,312,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,has_superstructure_mud_mortar_stone,0,0,1,653.306926
4,120101000051,1,1,30,308,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,has_superstructure_mud_mortar_stone,0,0,1,653.306926
5,120101000061,1,1,18,216,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,has_superstructure_mud_mortar_stone,0,0,1,653.306926
7,120101000081,1,1,8,280,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,has_superstructure_mud_mortar_stone,0,0,1,653.306926


In [35]:
# Check for NaN values
nan_data = df[df.isna().any(axis=1)]
if not nan_data.empty:
    nan_data.head()

In [ ]:
num_rows = df.shape[0]
print("Number of rows:", num_rows)

Number of rows: 755972


In [ ]:
# Check Data
df.head()

,building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,...,ground_floor_type,other_floor_type,position,plan_configuration,condition_post_eq,damage_grade,technical_solution_proposed,superstructure,floors_change,height_change
0,120101000011,12,1207,120703,1,1,9,288,9,9,...,Mud,Not applicable,Not attached,Rectangular,Damaged-Used in risk,Grade 3,Major repair,has_superstructure_mud_mortar_stone,0,0
1,120101000021,12,1207,120703,1,1,15,364,9,9,...,Mud,Not applicable,Not attached,Rectangular,Damaged-Repaired and used,Grade 5,Reconstruction,has_superstructure_mud_mortar_stone,0,0
2,120101000031,12,1207,120703,1,1,20,384,9,9,...,Mud,Not applicable,Not attached,Rectangular,Damaged-Repaired and used,Grade 2,Minor repair,has_superstructure_mud_mortar_stone,0,0
3,120101000041,12,1207,120703,1,1,20,312,9,9,...,Mud,Not applicable,Not attached,Rectangular,Damaged-Repaired and used,Grade 2,Minor repair,has_superstructure_mud_mortar_stone,0,0
4,120101000051,12,1207,120703,1,1,30,308,9,9,...,Mud,Not applicable,Not attached,Rectangular,Damaged-Repaired and used,Grade 1,Minor repair,has_superstructure_mud_mortar_stone,0,0


In [ ]:
# Save the cleaned data
df.to_csv('Data/eq2015_cleaned.csv',index=True)

In [ ]:
negative_values = df[df['floors_change'] < 0]
negative_values.head(20)

,building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,...,ground_floor_type,other_floor_type,position,plan_configuration,condition_post_eq,damage_grade,technical_solution_proposed,superstructure,floors_change,height_change
